In [1]:
import numpy as np
import random

In [2]:
m = np.array([[0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1.],
       [1., 1., 0., 0., 1.]])
print(m.shape)
m

(5, 5)


array([[0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1.],
       [1., 1., 0., 0., 1.]])

In [3]:
# get indices of all nonzero positions
nonzero_pos = np.nonzero(m)
print(nonzero_pos)
num_nonzero = len(nonzero_pos[0])



(array([0, 0, 0, 1, 1, 2, 3, 3, 4, 4, 4]), array([2, 3, 4, 3, 4, 4, 0, 4, 0, 1, 4]))


In [5]:
img_rows = m.shape[0]
img_cols = m.shape[1]

# select anchor example
# select any segment position at random
rand_anchor_idx = np.random.randint(num_nonzero)
rand_anchor_idx = 3 # temp
i = nonzero_pos[0][rand_anchor_idx]
j = nonzero_pos[1][rand_anchor_idx]
anchor = [i,j]
# print(f'i,j:{i,j}')

# select hard negative example
hard_negative = None
radius=1
while hard_negative is None:
    # find closest background pixel to anchor to be the hard negative
    # option 1 - check for background pixels using increasing radius approach
    square_length = 3 + 2 * (radius - 1)
#     print(f'radius:{radius}')
#     print(f'square_length:{square_length}')

    # get upper left corner of pixel at given radius from (i,j) even if illegal position
    ul_i = i - radius
    ul_j = j - radius
#     print(f'ul_i, ul_j:{ul_i, ul_j}')

    # calc all positions in radius-length square starting from that upper left position
    # ignore any positions that are segment pixels or illegal positions
    candidate_pos = [] 
    for a in range(square_length):
        for b in range(square_length):
            if a == 0 or a == square_length-1:
                pos = [ul_i+a,ul_j+b]
    #             print(f'pos:{pos}')
                c1 = pos[0] >= 0 and pos[0] < img_rows
                c2 = pos[1] >= 0 and pos[1] < img_cols

                if c1 and c2:

                    # check if position is a segment pixel
                    # get all segment row positions that match pos[0]
                    c3 = True
                    matches_row_idx = np.where(nonzero_pos[0] == pos[0])[0]
    #                 print(matches_row_idx)

                    for each in matches_row_idx:
    #                     print(f'nonzero_pos[1][each]:{nonzero_pos[1][each]}')
                        if nonzero_pos[1][each] == pos[1]:
    #                         print(f'segment position:{pos}')
                            c3 = False


                    if c3:
                        candidate_pos.append(pos)
            else:
                if b == 0 or b == square_length-1: 
                    pos = [ul_i+a,ul_j+b]
        #             print(f'pos:{pos}')
                    c1 = pos[0] >= 0 and pos[0] < img_rows
                    c2 = pos[1] >= 0 and pos[1] < img_cols

                    if c1 and c2:

                        # check if position is a segment pixel
                        # get all segment row positions that match pos[0]
                        c3 = True
                        matches_row_idx = np.where(nonzero_pos[0] == pos[0])[0]
        #                 print(matches_row_idx)

                        for each in matches_row_idx:
        #                     print(f'nonzero_pos[1][each]:{nonzero_pos[1][each]}')
                            if nonzero_pos[1][each] == pos[1]:
        #                         print(f'segment position:{pos}')
                                c3 = False


                        if c3:
                            candidate_pos.append(pos)
    if len(candidate_pos) > 0:
        # pick any candidate position as a hard negative because it is a closest background pixel to the anchor
        hard_negative = random.choice(candidate_pos)
    else:
        radius += 1
        

# select positive example
# choose any segment pixel except anchor as positive
rand_positive_idx=np.random.randint(num_nonzero)
while rand_positive_idx == rand_anchor_idx:
    rand_positive_idx=np.random.randint(num_nonzero)
    
i = nonzero_pos[0][rand_positive_idx]
j = nonzero_pos[1][rand_positive_idx]
positive = [i,j]


print(f'anchor:{anchor}')
print(f'positive:{positive}')
print(f'hard_negative:{hard_negative}')

anchor:[1, 3]
positive:[0, 4]
hard_negative:[2, 2]
